In [54]:
import toml

class configServiceImpl:
    
    @staticmethod
    def load_config_impl(config_file_path):
        with open(config_file_path, "r") as file:
            return toml.load(file)
        
    @staticmethod
    def get_config_value_impl(config, key):
        if config is not None:
            keys = key.split('.')
            for k in keys:
                config = config[k]
            return config
        else:
            return None
        
    @staticmethod
    def set_config_value_impl(self, key, new_value, config):
        keys = key.split('.')
        for k in keys[:-1]:
            config = config[k]
        config[keys[-1]] = new_value
        
    @staticmethod
    def save_config_impl(config_file_path, config):
        with open(config_file_path, "w") as file:
            toml.dump(config, file)


In [55]:
class Config:
    
    def __init__(self, config_file_path):
        self.config_file_path = config_file_path
        self.config_loader = configServiceImpl()
        self.config = self.load_config(config_file_path)
        
    def load_config(self, config_file_path):
        return self.config
    
    def get_config_value(self, key):
        pass
    
    def set_config_value(self, key, new_value):
        pass
    
    def save_config(self):
        pass

In [56]:
class ConfigService:
    
    def __init__(self, config_file_path):
        self.config_file_path = config_file_path
        self.config_loader = configServiceImpl()
        self.config = self.load_config(config_file_path)

    def load_config(self, config_file_path):
        try:
            return self.config_loader.load_config_impl(config_file_path)
            #with open(config_file_path, "r") as file:
            #    config_data = toml.load(file)
            #    return config_data
        except FileNotFoundError:
            raise FileNotFoundError(f"File di configurazione non trovato: {config_file_path}")
        except toml.TomlDecodeError as e:
            raise Exception(f"Errore durante il caricamento del file di configurazione: {str(e)}")

    def get_config_value(self, key):
        """
        Restituisce il valore associato a una chiave specifica nel file di configurazione.
        Se la chiave non è presente, restituisce None.
        """
        try:
            return configServiceImpl.get_config_value_impl(self.config, key)
            #keys = key.split('.')
            #config = self.config  # Aggiunto questo per mantenere l'originale
            #for k in keys:
            #    config = config[k]
            #return config
        except (TypeError, KeyError):
            return None

    def set_config_value(self, key, new_value):
        """
        Imposta il nuovo valore associato a una chiave specifica nel file di configurazione.
        """
        try:
            configServiceImpl.set_config_value_impl(self, key, new_value, self.config)
        except (TypeError, KeyError):
            # Se la chiave non esiste, puoi gestire l'errore qui a seconda dei requisiti.
            raise KeyError(f"La chiave '{key}' non esiste nel file di configurazione.")

        # Salva le modifiche nel file
        self.save_config()

    def save_config(self):
        """
        Salva le modifiche al file di configurazione.
        """
        try:
            configServiceImpl.save_config_impl(self.config_file_path, self.config)
            #with open(self.config_file_path, "w") as file:
            #    toml.dump(self.config, file)
        except Exception as e:
            raise Exception(f"Errore durante il salvataggio del file di configurazione: {str(e)}")


In [57]:
class Workflow:
    
    def __init__(self, eval_metric: str, decimal_precision: int, config_service: ConfigService):
        self.config_service = config_service
        self.eval_metric = eval_metric
        self.decimal_precision = decimal_precision

    def get_eval_metric(self):
        return self.eval_metric

    def get_decimal_precision(self):
        return self.decimal_precision


In [58]:
def setup(config_file_path):
    # Creazione di un'istanza di ConfigService
    config_service = ConfigService(config_file_path)

    # Creazione di un'istanza di Workflow con ConfigService come parametro
    workflow_instance = Workflow(
        eval_metric=config_service.get_config_value('bayesian_optimization.setup.eval_metric'),
        decimal_precision=config_service.get_config_value('bayesian_optimization.setup.decimal_precision'),
        config_service=config_service
        )
    
    # Utilizzo del metodo get_eval_metric
    def get_eval_metrics():
        eval_metric_value = workflow_instance.get_eval_metric()
        return eval_metric_value
    #eval_metric_value = workflow_instance.get_eval_metric()
    decimal_precision_value = workflow_instance.get_decimal_precision()
    
    # Stampa del valore ottenuto
    print(f"Valore di eval_metric: {get_eval_metrics()}")
    print(f"Valore di decimal_precision: {decimal_precision_value}")

# Chiamata alla funzione setup
setup("/work/asc/machine_learning/projects/iMagine/bayes_opt/src/data_connection/workflow_config.toml")



Valore di eval_metric: FSS
Valore di decimal_precision: 4


In [12]:
def new_value():
    return [0.0133432423, 11.62423432432, 15.364234324]

def mod_config(values):
    
    # Creazione di un'istanza di ConfigService
    config_service = ConfigService("/work/asc/machine_learning/projects/iMagine/bayes_opt/src/data_connection/workflow_config.toml")
    
    decimal_precision=config_service.get_config_value('bayesian_optimization.setup.decimal_precision')
    
    # Arrotonda i valori a due decimali
    rounded_values = [round(val, decimal_precision) for val in values]

    # Impostare un nuovo valore per la chiave 'medslik2.sim_params.v'
    config_service.set_config_value('medslik2.sim_params.v', rounded_values)

    # Verifica del nuovo valore
    new_value = config_service.get_config_value('medslik2.sim_params.v')
    print(f"Nuovo valore di v: {new_value}")

mod_config(new_value())

Nuovo valore di v: [0.0133, 11.6242, 15.3642]


In [13]:
class Bay_Opt_Setup:
    
    def __init__(self, eval_metric: str, init_points: int, n_iter: int,
                 random_state: int, decimal_precision: int, verbose: int,
                 config_service = ConfigService):
        
        self._eval_metric = eval_metric
        self._init_points = init_points
        self._n_iter = n_iter
        self._random_state = random_state
        self._decimal_precision = decimal_precision
        self._verbose = verbose
        self.config_service = config_service

    @property
    def get_eval_metric(self):
        return self._eval_metric
    
    @get_eval_metric.setter
    def set_eval_metric(self, value):
        self._eval_metric = value

    @property
    def get_init_points(self):
        return self._init_points
    
    @get_init_points.setter
    def set_init_points(self, value):
        self._init_points = value

    @property
    def get_n_iter(self):
        return self._n_iter
    
    @get_n_iter.setter
    def set_n_iter(self, value):
        self._n_iter = value

    @property
    def get_random_state(self):
        return self._random_state
    
    @get_random_state.setter
    def set_random_state(self, value):
        self._random_state = value

    @property
    def get_decimal_precision(self):
        return self._decimal_precision
    
    @get_decimal_precision.setter
    def set_decimal_precision(self, value):
        self._decimal_precision = value

In [14]:
class Bay_Opt_Setup_Service:
    
    def __init__(self, config_file_path):
        # Creazione di un'istanza di ConfigService
        self.config_service = ConfigService(config_file_path)

        # Creazione di un'istanza di Bay_Opt_Setup con ConfigService come parametro
        self.bay_opt_setup_instance = Bay_Opt_Setup(
            eval_metric=self.config_service.get_config_value('bayesian_optimization.setup.eval_metric'),
            init_points=self.config_service.get_config_value('bayesian_optimization.setup.init_points'),
            n_iter=self.config_service.get_config_value('bayesian_optimization.setup.n_iter'),
            random_state=self.config_service.get_config_value('bayesian_optimization.setup.random_state'),
            decimal_precision=self.config_service.get_config_value('bayesian_optimization.setup.decimal_precision'),
            verbose=self.config_service.get_config_value('bayesian_optimization.setup.verbose'),
            config_service=self.config_service
        )
    
    # Utilizzo del metodo get_eval_metric
    def get_eval_metrics(self):
        return self.bay_opt_setup_instance.get_eval_metric
        
    # Metodo per ottenere decimal_precision
    def get_decimal_precision_value(self):
        return self.bay_opt_setup_instance.get_decimal_precision
    
    # Stampa del valore ottenuto
    #print(f"Valore di eval_metric: {get_eval_metrics()}")
    #print(f"Valore di decimal_precision: {get_decimal_precision()}")

# Chiamata alla funzione setup
#setup("/work/asc/machine_learning/projects/iMagine/bayes_opt/src/data_connection/workflow_config.toml")


In [15]:
# Creare un'istanza di Bay_Opt_Service
bay_opt_service_instance = Bay_Opt_Setup_Service("/work/asc/machine_learning/projects/iMagine/bayes_opt/src/data_connection/workflow_config.toml")


# Ottenere il valore di eval_metrics
eval_metrics_value = bay_opt_service_instance.get_eval_metrics()
decimal_precision_value = bay_opt_service_instance.get_decimal_precision_value()
print(f"Valore di eval_metrics: {eval_metrics_value}")
print(decimal_precision_value)


Valore di eval_metrics: FSS
4


In [41]:
import typing as t

class Bay_Opt_Bounds:
    
    def __init__(self, b: t.Dict,
                 config_service = ConfigService):      
        self._b = b
        self.config_service = config_service

    @property
    def get_b(self):
        return self._b
    
    @get_b.setter
    def set_b(self, value: t.Dict):
        self._b = value

In [122]:
class Bay_Opt_Bounds_Service:
    
    def __init__(self, config_file_path):
        self.b = b
        self.bay_opt_bounds_service = Bay_Opt_Bounds_Service_Impl()
        # Creazione di un'istanza di ConfigService
        self.config_service = ConfigService(config_file_path)

        # Creazione di un'istanza di Bay_Opt_Setup con ConfigService come parametro
        self.bay_opt_bounds_instance = Bay_Opt_Bounds(
            b = self.config_service.get_config_value('bayesian_optimization.b'),
            config_service=self.config_service
        )
        
    def get_b(self):
        try:
            return self.bay_opt_bounds_instance.get_b
        except (TypeError, KeyError):
            return None
        
    def set_b(self, value: t.Dict):
        try:
            # Ottieni il valore corrente di 'b'
            valore_corrente_di_b = self.bay_opt_bounds_instance.get_b()

            # Utilizza il metodo set_b_value_impl per aggiornare il valore di 'b'
            self.bay_opt_bounds_service.set_b_value_impl(valore_corrente_di_b, 'bayesian_optimization.b', value)

            # Ottieni il valore aggiornato di 'b'
            valore_aggiornato_di_b = self.bay_opt_bounds_instance.get_b()
            return valore_aggiornato_di_b
        except (TypeError, KeyError):
            # Se la chiave non esiste, puoi gestire l'errore qui a seconda dei requisiti.
            raise KeyError(f"La chiave non esiste nel file di configurazione.")
        
        

In [123]:
class Bay_Opt_Bounds_Service_Impl():
    
    @staticmethod
    def set_b_value_impl():
        try:
            config_service_instance = configServiceImpl()
            config_service_instance.set_config_value_impl(key, new_value, config)
            config_service_instance.save_config_impl("/work/asc/machine_learning/projects/iMagine/bayes_opt/src/data_connection/workflow_config.toml", config)
        except (TypeError, KeyError):
            raise KeyError(f"La chiave '{key}' non esiste nel file di configurazione.")


In [124]:
# Creare un'istanza di Bay_Opt_Bounds_Service
bay_opt_bounds_service_instance = Bay_Opt_Bounds_Service("/work/asc/machine_learning/projects/iMagine/bayes_opt/src/data_connection/workflow_config.toml")

# Ottenere il valore corrente di 'b'
valore_corrente_di_b = bay_opt_bounds_service_instance.get_b()
print("Valore corrente di 'b':", valore_corrente_di_b)

# Creare un nuovo valore per 'b'
nuovo_valore_di_b = {"b": [[0.0, 0.03], [0.0, 17.0], [0.0, 21.0]]}

# Utilizzare il metodo set_b per aggiornare il valore di 'b'
valore_aggiornato_di_b = bay_opt_bounds_service_instance.set_b(nuovo_valore_di_b)

# Stampa il valore aggiornato di 'b'
print("Valore aggiornato di 'b':", valore_aggiornato_di_b)


Valore corrente di 'b': [[0.0, 0.05], [0.0, 15.0], [0.0, 20.0]]


AttributeError: 'ConfigService' object has no attribute 'load_config_impl'

In [115]:
# Creare un'istanza di Bay_Opt_Service
bay_opt_bounds_instance = Bay_Opt_Bounds_Service("/work/asc/machine_learning/projects/iMagine/bayes_opt/src/data_connection/workflow_config.toml")


# Ottenere il valore di eval_metrics
b = bay_opt_bounds_instance.get_b()
print(f"Valore di b: {b}")

Valore di b: None


In [90]:
config_service_instance = configServiceImpl()  # Definire config_service_instance come variabile globale

def new_values_dict():
    return [[0.0, 0.03], [0.0, 17.0], [0.0, 21.0]]

def mod_config_dict(values):
    global config_service_instance  # Dichiarare config_service_instance come globale
    # Creare un'istanza di Bay_Opt_Bounds_Service
    bay_opt_bounds_service_instance = Bay_Opt_Bounds_Service("/work/asc/machine_learning/projects/iMagine/bayes_opt/src/data_connection/workflow_config.toml")

    # Ottenere il valore corrente di 'b'
    valore_corrente_di_b = bay_opt_bounds_service_instance.get_b()
    print("Valore corrente di 'b':", valore_corrente_di_b)

    # Creare un nuovo valore per 'b'
    nuovo_valore_di_b = {"b": values}

    # Utilizzare il metodo set_b per aggiornare il valore di 'b'
    bay_opt_bounds_service_instance.set_b('bayesian_optimization.b', nuovo_valore_di_b)

    # Ottenere il valore aggiornato di 'b'
    valore_aggiornato_di_b = bay_opt_bounds_service_instance.get_b
    print("Valore aggiornato di 'b':", valore_aggiornato_di_b)

# Utilizzare il metodo mod_config_dict con i nuovi valori
mod_config_dict(new_values_dict())


Valore corrente di 'b': [[0.0, 0.05], [0.0, 15.0], [0.0, 20.0]]


KeyError: "La chiave 'bayesian_optimization.b' non esiste nel file di configurazione."

In [ ]:
def new_value():
    return [[0.0, 0.03], [0.0, 17.0], [0.0, 21.0]]

def mod_config(values):
    
    # Creazione di un'istanza di ConfigService
    config_service = ConfigService("/work/asc/machine_learning/projects/iMagine/bayes_opt/src/data_connection/workflow_config.toml")
    
    decimal_precision=config_service.get_config_value('bayesian_optimization.setup.decimal_precision')
    
    # Arrotonda i valori a due decimali
    rounded_values = [round(val, decimal_precision) for val in values]

    # Impostare un nuovo valore per la chiave 'medslik2.sim_params.v'
    config_service.set_config_value('medslik2.sim_params.v', rounded_values)

    # Verifica del nuovo valore
    new_value = config_service.get_config_value('medslik2.sim_params.v')
    print(f"Nuovo valore di v: {new_value}")

mod_config(new_value())